<a href="https://colab.research.google.com/github/apoorvapu/data_science/blob/main/NLP_languageTranslation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**NLP**: **(2 methods: (1) model training from scratch and (2) fine-tuning LLM)**

#**(1) model training from scratch**

In [23]:
from google.colab import drive
drive.mount('/content/drive')
# Read the API key from the file
with open("/content/drive/My Drive/hf_token.txt", "r") as f:
    hf_token = f.read().strip()

from huggingface_hub import login
login(hf_token)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [24]:
# Read the API key from the file
with open('/content/drive/MyDrive/wandb_key.txt', 'r') as f:
    wandb_key = f.read().strip()

# Set the W&B API key
import os
os.environ["WANDB_API_KEY"] = wandb_key

In [25]:
!pip install datasets bitsandbytes accelerate transformers sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 4.1 MB/s eta 0:00:00


In [26]:
from transformers import Trainer, TrainingArguments
from peft import get_peft_model, LoraConfig, TaskType
import torch, gc
from datasets import load_dataset
torch.cuda.empty_cache()
gc.collect()

194

In [27]:
# Load dataset
dataset = load_dataset("rahular/itihasa")  # Example: sanskrit-eng translation

In [28]:
import gc
train_dataset = dataset["train"]
val_dataset = dataset["validation"]
test_dataset = dataset["test"]
del dataset
gc.collect()

54

In [29]:
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

def preprocess_function(examples):
    # Extract English ('en') and Sanskrit ('sn') texts from the 'translation' field in the list
    inputs = tokenizer([example['en'] for example in examples['translation']], padding="max_length", truncation=True, max_length=64)
    labels = tokenizer([example['sn'] for example in examples['translation']], padding="max_length", truncation=True, max_length=64)

    # Add the labels as the 'input_ids' of the Sanskrit tokens
    inputs['labels'] = labels['input_ids']
    return inputs


In [30]:
train_dataset = train_dataset.map(preprocess_function, batched=True)
val_dataset = val_dataset.map(preprocess_function, batched=True)
test_dataset = test_dataset.map(preprocess_function, batched=True)

In [31]:
from transformers import T5Config, T5ForConditionalGeneration

# Define model configuration (smaller for faster training)
config = T5Config(
    vocab_size=tokenizer.vocab_size,
    d_model=256,  # Smaller model size (default is 512 or more)
    num_layers=4,  # Reduce layers for efficiency
    num_heads=4,  # Reduce attention heads
    d_ff=1024,  # Feed-forward size
    dropout_rate=0.1
)

# Initialize the model
model = T5ForConditionalGeneration(config)


In [32]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./transformer_scratch",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    learning_rate=3e-4,
    num_train_epochs=5,
    weight_decay=0.01,
    save_total_limit=2,
    logging_dir="./logs",
    logging_steps=500,
    push_to_hub=False
)

# Define Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
)


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-32-b085b44d9e49>:19: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [33]:
trainer.train()


ValueError: self.model.config.decoder_start_token_id has to be defined. In T5 it is usually set to the pad_token_id. See T5 docs for more information.

In [ ]:
# Evaluate on validation set
results = trainer.evaluate()
print("Evaluation results:", results)

# Save the model
model.save_pretrained("./transformer_scratch")
tokenizer.save_pretrained("./transformer_scratch")


In [ ]:
def translate(text):
    inputs = tokenizer(text, return_tensors="pt").input_ids
    outputs = model.generate(inputs, max_length=128, num_beams=5)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

print(translate("I am happy today."))  # Should return Sanskrit translation


# **(2) fine-tuning LLM**

# language translation

For fine-tuning Meta-llama2 for language translation with low memory,you can use techniques like LoRA, QLoRA, 8-bit/4-bit quantization, and Retrieval-Augmented Generation (RAG).

In [ ]:
from transformers import LlamaForCausalLM, LlamaTokenizer, Trainer, TrainingArguments
from peft import get_peft_model, LoraConfig, TaskType
import torch, gc
from datasets import load_dataset
torch.cuda.empty_cache()
gc.collect()

In [ ]:
# Check if CUDA (GPU support) is available
cuda_available = torch.cuda.is_available()
print("CUDA Available:", cuda_available)

# Get the name of the GPU being used
gpu_name = torch.cuda.get_device_name(0) if cuda_available else "No GPU"
print("GPU Name:", gpu_name)

In [ ]:
# Load dataset
dataset = load_dataset("rahular/itihasa")  # Example: sanskrit-eng translation

In [ ]:
from transformers import BitsAndBytesConfig

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True
)

In [ ]:
'''
# Load tokenizer and model
model_name = "meta-llama/Llama-2-7b-hf"
tokenizer = LlamaTokenizer.from_pretrained(model_name)
model = LlamaForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16)
'''

In [ ]:
'''
# Load model directly
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("facebook/mbart-large-50")
model = AutoModelForSeq2SeqLM.from_pretrained("facebook/mbart-large-50", quantization_config=quantization_config)
'''

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("facebook/nllb-200-distilled-600M")
model = AutoModelForSeq2SeqLM.from_pretrained("facebook/nllb-200-distilled-600M")

In [ ]:
print(dataset)

In [ ]:
dataset['train'][0]

In [ ]:
import gc
train_dataset = dataset["train"]
val_dataset = dataset["validation"]
test_dataset = dataset["test"]
del dataset
gc.collect()

In [ ]:
'''
import random
from random import sample

# Convert dataset to a list (this works for Hugging Face Datasets)
dataset_list = list(train_dataset)
# Now you can sample a fraction of it, for example, 20%
train_dataset = random.sample(dataset_list, int(len(dataset_list) * 0.1))

dataset_list = list(val_dataset)
# Now you can sample a fraction of it, for example, 20%
val_dataset = random.sample(dataset_list, int(len(dataset_list) * 0.2))
'''

In [ ]:
print(train_dataset)

In [ ]:
train_dataset[0]

In [ ]:
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

def preprocess_function(examples):
    # Extract English ('en') and Sanskrit ('sn') texts from the 'translation' field in the list
    inputs = tokenizer([example['en'] for example in examples['translation']], padding="max_length", truncation=True, max_length=64)
    labels = tokenizer([example['sn'] for example in examples['translation']], padding="max_length", truncation=True, max_length=64)

    # Add the labels as the 'input_ids' of the Sanskrit tokens
    inputs['labels'] = labels['input_ids']
    return inputs


In [ ]:
train_dataset = train_dataset.map(preprocess_function, batched=True)
val_dataset = val_dataset.map(preprocess_function, batched=True)
test_dataset = test_dataset.map(preprocess_function, batched=True)

In [ ]:
torch.cuda.empty_cache()
gc.collect()

In [ ]:
# Define LoRA configuration
lora_config = LoraConfig(
    r=2,  # Rank for LoRA
    lora_alpha=8,  # Scaling factor for LoRA
    target_modules=["q_proj", "v_proj"],  # Modules to apply LoRA to
    lora_dropout=0.1,  # Dropout for LoRA layers
    task_type=TaskType.SEQ_2_SEQ_LM,
    bias="none",  # Freeze biases
)

# Apply LoRA to the model
model = get_peft_model(model, lora_config)
device = "cuda" if torch.cuda.is_available() else "cpu"
# Move your model to the GPU
model.to(device)

In [ ]:
torch.cuda.empty_cache()
gc.collect()

In [ ]:
from transformers import EarlyStoppingCallback

# Define the training arguments
training_args = TrainingArguments(
    output_dir="./results",  # Directory to save results and model checkpoints
    evaluation_strategy="steps",  # Evaluation strategy (every `eval_steps` steps)
    save_strategy="steps",  # Save checkpoint every `save_steps`
    save_steps=500,  # Save model every 500 steps (adjust as needed)
    save_total_limit=2,  # Keep the latest 2 checkpoints
    logging_dir="./logs",  # Where to save logs
    logging_steps=500,  # Log every 500 steps
    per_device_train_batch_size=20,  # Adjust based on GPU memory
    gradient_accumulation_steps=4,  # Simulate larger batch sizes
    fp16=True,  # Enable mixed precision for memory optimization
    load_best_model_at_end=True,  # Automatically load the best model at the end of training
    max_steps=1000,  # Train for exactly 10,000 steps
    eval_steps=500,  # Evaluate every 1000 steps
    dataloader_num_workers=4,  # Parallelize data loading
)

# Early stopping callback
callbacks = [EarlyStoppingCallback(early_stopping_patience=3)]

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    callbacks=callbacks,  # Add early stopping callback
)

# Start training
trainer.train()


In [ ]:
# Evaluate the model on the test dataset
results = trainer.evaluate(test_dataset)

# Print the evaluation results
print(f"Evaluation results on the test dataset: {results}")


In [ ]:
model.save_pretrained("./fine_tuned_translation_model")
tokenizer.save_pretrained("./fine_tuned_translation_model")


In [ ]:
del train_dataset, val_dataset, results
gc.collect()
torch.cuda.empty_cache()

In [ ]:
def hi(examples):
    # Tokenize the English sentences (inputs) and return tensors
    inputs = tokenizer(examples['text'], padding="max_length", max_length=64, return_tensors="pt")
    return inputs

# Data: English sentences
data = ["I am very happy today and hungry too.", "Life is good.", "All is well."]

# Wrap it in a dataset
dataset = Dataset.from_dict({"text": data})

# Apply the preprocessing function
dataset = dataset.map(hi, batched=True)

# Get inputs from the dataset
inputs = dataset['input_ids']  # Extract the input ids after preprocessing

# Convert the list of input_ids into a tensor and move to the appropriate device
inputs_tensor = torch.tensor(inputs).to(device)  # Move to device (GPU or CPU)

# Move the model to the device
model.to(device)

# Make predictions
with torch.no_grad():
    outputs = model.generate(
    input_ids=inputs_tensor,
    max_length=128,
    num_beams=5,
    temperature=0.7,  # Adjust temperature for diversity
    top_p=0.9,  # nucleus sampling
    top_k=50,  # Limit the top-k tokens to sample from
    early_stopping=True
    )

# Decode the predictions (Sanskrit translations)
decoded_preds = tokenizer.batch_decode(outputs, skip_special_tokens=True)

# Print the translations
for i, text in enumerate(decoded_preds):
    print(f"English: {data[i]}")
    print(f"Sanskrit Translation: {text}")
    print('-' * 50)